In [1]:
import pandas as pd
import json
from IPython.display import display, HTML
display(HTML("<style>.container{width: 100%}</style>"))

In [2]:
df_raw = pd.read_parquet("./.data/transactions.parquet")
dC_raw = pd.read_csv("./.data/clients.csv")
dP_raw = pd.read_csv("./.data/plants.csv")
dM_raw = pd.read_csv("./.data/materials.csv")

In [3]:
ids = json.load(open('ids.json', 'r'))

In [4]:
def process_transactions(df):
    df = df[df["sales_sum"] >= 0]
    df = df[df["plant"].isin(ids["selected_plants"])]
    chq_sum = df.groupby("chq_id").agg(
        chq_sum=pd.NamedAgg("sales_sum", "sum")
    ).reset_index()
    df = pd.merge(chq_sum, df, how='inner', on="chq_id")
    mat_stat = df.groupby(["material", "plant", "chq_date"]).agg(
        material_chq_count=pd.NamedAgg("chq_id", "nunique"),
        material_clients_count=pd.NamedAgg("client_id", "nunique"),
        material_chq_sum=pd.NamedAgg("chq_sum", "sum"),
        material_sales_count=pd.NamedAgg("sales_count", "sum"),
        material_sales_sum=pd.NamedAgg("sales_sum", "sum")
    ).reset_index()
    cli_stat = df.groupby(["client_id", "plant", "chq_date"]).agg(
        client_chq_count=pd.NamedAgg("chq_id", "nunique"),
        client_chq_sum=pd.NamedAgg("chq_sum", "sum"),
        client_sales_count=pd.NamedAgg("sales_count", "count"),
        client_promo_count=pd.NamedAgg("is_promo", "sum"),
    ).reset_index()
    df = pd.merge(mat_stat, df, how='inner', on=["material", "plant", "chq_date"])
    df = pd.merge(cli_stat, df, how='inner', on=["client_id", "plant", "chq_date"])
    return df, mat_stat, cli_stat
# _df_raw = df_raw.head(40000)
# _df = process_transactions(_df_raw)
df, dMstat, dCstat = process_transactions(df_raw)

In [5]:
display(dP_raw.shape, dC_raw.shape, dM_raw.shape, dMstat.shape, dCstat.shape, df.shape)
display(dP_raw.head())
display(dC_raw.head())
display(dM_raw.head())
display(dMstat.head())
display(dCstat.head())
display(df.head())

(388, 3)

(99995, 4)

(105609, 8)

(2196358, 8)

(176119, 7)

...

,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,M,Other,1990.0
1,9cb909f701d25d548e953bff81192b56,F,Other,1969.0
2,d5da0f5b841b4f83383202807995027a,M,Other,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,F,Moscow,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,F,Other,1988.0


,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0


,material,plant,chq_date,material_chq_count,material_clients_count,material_chq_sum,material_sales_count,material_sales_sum
0,00011c1252f877221e0e0a3ddf21858e,0e7e3cf0ded4d9db8b376b317c007f99,2017-04-02,1,1,6007.0,1.0,119.99
1,00011c1252f877221e0e0a3ddf21858e,0e7e3cf0ded4d9db8b376b317c007f99,2017-04-03,1,1,12019.0,1.0,119.99
2,00011c1252f877221e0e0a3ddf21858e,2ebe25dd3a566f36f80d55440d3c3834,2017-02-07,1,1,1108.0,1.0,119.99
3,00011c1252f877221e0e0a3ddf21858e,6950aac2d7932e1f1a4c3cf6ada1316e,2017-04-16,1,1,3793.0,1.0,228.39
4,00011c1252f877221e0e0a3ddf21858e,7f8bb0fe8b33780a08fe6b60ced14529,2017-02-23,1,1,472.0,1.0,119.56


,client_id,plant,chq_date,client_chq_count,client_chq_sum,client_sales_count,client_promo_count
0,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4
1,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-11-20,1,37125.0,15,6
2,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-01,1,89620.0,20,20
3,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-13,1,16522.0,11,11
4,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-27,1,25.0,1,0


,client_id,plant,chq_date,client_chq_count,client_chq_sum,client_sales_count,client_promo_count,material,material_chq_count,material_clients_count,material_chq_sum,material_sales_count,material_sales_sum,chq_id,chq_sum,chq_position,sales_count,sales_sum,is_promo
0,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4,00c5d51c9029b72cfe32b3935f4685ac,1,1,1655.0,1.0,349.98,5e58f7ad4e48291073ee4807d952dd62,1655.0,002,1.0,349.98,1
1,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4,0d08064996191325836d2a1e2bd1d660,3,3,7606.0,3.0,31.82,5e58f7ad4e48291073ee4807d952dd62,1655.0,006,1.0,10.99,0
2,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4,40f96e886f80e25f241b1781a2543157,1,1,1655.0,2.0,105.58,5e58f7ad4e48291073ee4807d952dd62,1655.0,009,2.0,105.58,0
3,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4,4388eb5d517c775afd29e27d63752135,1,1,1655.0,1.0,56.49,5e58f7ad4e48291073ee4807d952dd62,1655.0,003,1.0,56.49,0
4,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,1,19860.0,12,4,564840544bef2c259cf7a0642ab97b2e,2,2,2797.0,3.0,128.85,5e58f7ad4e48291073ee4807d952dd62,1655.0,005,2.0,85.86,0


In [6]:
df.sort_values("material_chq_count", ascending=False).head()

,client_id,plant,chq_date,client_chq_count,client_chq_sum,client_sales_count,client_promo_count,material,material_chq_count,material_clients_count,material_chq_sum,material_sales_count,material_sales_sum,chq_id,chq_sum,chq_position,sales_count,sales_sum,is_promo
1839342,a6f454789e130da55177e807443b38ee,43692f46e3168b32434dd507ebc85dbe,2017-04-03,1,1880.0,4,3,3afa2c41341548499934a52d7338f33e,52,50,200837.0,116.0,807.81,1eca8d411eb02483892d7db73ff877f4,470.0,001,2.0,13.98,0
597629,354e3fe98d1a684a870f7e4c4c53edf4,43692f46e3168b32434dd507ebc85dbe,2017-04-03,1,425867.0,47,14,3afa2c41341548499934a52d7338f33e,52,50,200837.0,116.0,807.81,6737f96c7add39c44d2f7ae2b3eecec6,9061.0,013,6.0,41.94,0
1001458,5c184d1a05b58b2d9ab199b3d79b1a76,43692f46e3168b32434dd507ebc85dbe,2017-04-03,1,59508.0,18,12,3afa2c41341548499934a52d7338f33e,52,50,200837.0,116.0,807.81,ebe42baa8220183e87d4967746d1d770,3306.0,001,2.0,13.98,0
702257,3e277b148c386b611705b55b7edc4f38,43692f46e3168b32434dd507ebc85dbe,2017-04-03,1,48860.0,28,16,3afa2c41341548499934a52d7338f33e,52,50,200837.0,116.0,807.81,df98cc3d9a52c806986a5f2604520be9,1745.0,001,4.0,27.96,0
565787,328b3b14e0f794d8d7152a2a8809fc24,43692f46e3168b32434dd507ebc85dbe,2017-04-03,1,280830.0,46,26,3afa2c41341548499934a52d7338f33e,52,50,200837.0,116.0,807.81,37643bc67cce8ccdf3a7da7d27373493,6105.0,001,3.0,20.97,0


In [7]:
df["i_day_ratio"] = df["chq_sum"] / df["client_chq_sum"]

In [8]:
imp = df.groupby(["client_id", "material", "plant"]).agg(
    i_c=pd.NamedAgg("i_day_ratio", "mean"),
    including_chq_sum=pd.NamedAgg("chq_sum", "sum")
).reset_index().sort_values("i_c", ascending=False)
imp["i_cw"] = imp["i_c"]*imp["including_chq_sum"]
imp.head()

,client_id,material,plant,i_c,including_chq_sum,i_cw
884319,7c49213f0c6f71ae6e320146a4604e1a,6ca99c1547c41e1073730c35c77d1416,2ebe25dd3a566f36f80d55440d3c3834,1.0,171.0,171.0
1592476,dd6c35c5926c1851609fde73e3d74c4c,a35bd829438ec53b15c64211d4d99e44,7f8bb0fe8b33780a08fe6b60ced14529,1.0,178.0,178.0
282078,27b478b3c84bb03e01ecc69d9fb30cc1,2bb443370b6ff7bed42f85f77a8d6b41,29549a71a57f587d88209b9c1f1b7999,1.0,169.0,169.0
336812,2e94cde8e066b9f529ac4a46483be4e8,de2fe19a39ac21ac84d13a282d45bb8f,2ebe25dd3a566f36f80d55440d3c3834,1.0,145.0,145.0
547498,4cadd99bbd6c968c8d2e88543864145c,98d827281e9e513804c3a6570ad00644,6bcf53c2bdbee891ffaca1e612b0076c,1.0,49.0,49.0


In [15]:
fimp = imp.groupby(["material", "plant"]).agg(
    i_cw_sum=pd.NamedAgg("i_cw", "sum"),
    including_chq_sum_sum=pd.NamedAgg("including_chq_sum", "sum"),
).reset_index()
fimp["g_i_c"] = fimp["i_cw_sum"]/ fimp["including_chq_sum_sum"]
fimp = fimp.sort_values("g_i_c", ascending=False)
fimp.to_csv("./.data/imp.csv")
fimp.head()

,material,plant,i_cw_sum,including_chq_sum_sum,g_i_c
117630,66240a7af6515765a81b9e7d7c65f337,6950aac2d7932e1f1a4c3cf6ada1316e,1199.0,1199.0,1.0
153319,84b4b232ab542b4bf90405aaccffb1f3,6bcf53c2bdbee891ffaca1e612b0076c,1799.0,1799.0,1.0
199238,ac223b1a9922ec6debedc9a9eebc154b,43692f46e3168b32434dd507ebc85dbe,3999.0,3999.0,1.0
265900,e48a6716ee3ae20a8f85b0ecff59d98c,43692f46e3168b32434dd507ebc85dbe,178.0,178.0,1.0
12522,0b0fd926dbfbd4448bd5379cd95db6e7,7f8bb0fe8b33780a08fe6b60ced14529,353.0,353.0,1.0


In [11]:
imp_df = pd.merge(imp, df, how='inner', on=["client_id", "material", "plant"])
imp_df.head()
imp_df[['plant', 'client_id', 'chq_date', 'sales_count', 'sales_sum', 'material', "i_c"]].to_parquet("./.data/tr.parquet")

In [14]:
sat = imp_df.groupby(["client_id", "plant", "chq_date"]).agg(
    satisfaction=pd.NamedAgg("i_c", "sum")
).reset_index()
sat.to_csv("./.data/sat.csv")
sat.head()

,client_id,plant,chq_date,satisfaction
0,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-10-31,0.890957
1,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-11-20,0.969902
2,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-01,1.012593
3,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-13,0.898814
4,0000d48a8891cd462fdf01ea861ec53e,6858fb45a3d3aef7c29322d3b68dffd1,2016-12-27,1.000000


In [17]:
sat["satisfaction"].describe()

count    176119.000000
mean          1.000000
std           0.214187
min           0.046355
25%           0.905767
50%           1.000000
75%           1.083351
max           4.178344
Name: satisfaction, dtype: float64

In [18]:
imp.head()

,client_id,material,plant,i_c,including_chq_sum,i_cw
884319,7c49213f0c6f71ae6e320146a4604e1a,6ca99c1547c41e1073730c35c77d1416,2ebe25dd3a566f36f80d55440d3c3834,1.0,171.0,171.0
1592476,dd6c35c5926c1851609fde73e3d74c4c,a35bd829438ec53b15c64211d4d99e44,7f8bb0fe8b33780a08fe6b60ced14529,1.0,178.0,178.0
282078,27b478b3c84bb03e01ecc69d9fb30cc1,2bb443370b6ff7bed42f85f77a8d6b41,29549a71a57f587d88209b9c1f1b7999,1.0,169.0,169.0
336812,2e94cde8e066b9f529ac4a46483be4e8,de2fe19a39ac21ac84d13a282d45bb8f,2ebe25dd3a566f36f80d55440d3c3834,1.0,145.0,145.0
547498,4cadd99bbd6c968c8d2e88543864145c,98d827281e9e513804c3a6570ad00644,6bcf53c2bdbee891ffaca1e612b0076c,1.0,49.0,49.0
